## Proyecto B: Calculadora de ReTweets

Inicialmente, debemos construir un dataset con la mayor cantidad de tweets en Colombia durante la última semana. Para ello, seleccionamos un total de 100604 tweets de la API de Twitter publicados del 15 al 21 de Febrero.

In [10]:
# Libraries
from gensim.models.word2vec import Word2Vec
import pandas as pd
import numpy as np
import gradio as gr
import re
import nltk
from nltk import stem
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse.linalg import svds as SparseSVD

In [16]:
# Dataset
df = pd.concat([pd.read_csv(f'data/period{i}.csv') for i in range(7)])
df = df[["id","lang","tweet"]]
df.head()

,id,lang,tweet
0,1493736882106245120,es,@JuanCar99077589 @elojodiestro @intiasprilla Y...
1,1493736869108006912,es,@NairoQuinCo no tiene ni puta idea como es la ...
2,1493736868755611649,es,@Enrique_GomezM deje me decirle con todo res...
3,1493736858999787521,es,@PizarroMariaJo @MovimientoMAIS @UP_Colombia @...
4,1493736858769108992,es,Mal día para el duquecito en el parlamento eur...


### Análisis de Tópicos

In [17]:
def preprocessing(d):
    # d -> document
    words = d.split()
    
    # Remove special characters and punctuation
    proc1 = [re.sub('[^A-Za-z0-9]+', '', words[i]) for i in range(len(words))]
    
    # Remove empty strings
    proc2 = [w for w in proc1 if w != '']
    
    # Convert to lowercase every word  
    proc3 = [proc2[i].lower() for i in range(len(proc2))]
    
    # Remove stopwords
    proc4 = [w for w in proc3 if w not in ENGLISH_STOP_WORDS]

    # Lemmatization
    wnl = stem.WordNetLemmatizer()        
    proc5 = [wnl.lemmatize(w) for w in proc4]
    
    
    processed_string = ' '.join(proc5)
    return processed_string

num_top_words = 10

def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

In [18]:
# corpus -> set of documents
# document -> sentence (tweet)
# token -> word

tweets = df.tweet
tweets = [x.split() for x in tweets]
corpus = [' '.join(tweets[i]) for i in range(len(tweets))]

corpus

['@JuanCar99077589 @elojodiestro @intiasprilla Y evidentemente uds los narcotraficantes de la Bacrim \'Colombia Humana\' no piensan, ni siquiera notan la diferencia entre "ósea" y "o sea".',
 '@NairoQuinCo no tiene ni puta idea como es la vida de la mayoría de campesinos en Colombia. Miseria y humillación! https://t.co/8KGqPW9Ar7',
 '@Enrique_GomezM deje me decirle con todo respeto usted es un hta pero lo dos buenos . Con los pantalones bien puestos el q nesecitamos para el Colombia. Usted me representa !!!',
 '@PizarroMariaJo @MovimientoMAIS @UP_Colombia @PoloDemocratico @petrogustavo @LevyRincon @AquinoTicias1 @Betocoralg @ColombiaHumana_ @somospactometa @PactoCol Cuantos delincuentes en un sola imagen',
 'Mal día para el duquecito en el parlamento europeo. Le tocó quedarse calladito, creyó que ir de paseo por el mundo a nadie le iba importar lo que pasa en Colombia.',
 'Acaba de publicar una foto en Barranquilla, Atlantico, Colombia https://t.co/zhtQMk6MSV',
 'Zapatilla Converse Tal

In [19]:
corpus = [preprocessing(d) for d in corpus]
corpus

['juancar99077589 elojodiestro intiasprilla y evidentemente uds los narcotraficantes la bacrim colombia humana piensan ni siquiera notan la diferencia entre sea y o sea',
 'nairoquinco tiene ni puta idea como e la vida la mayora campesinos en colombia miseria y humillacin httpstco8kgqpw9ar7',
 'enriquegomezm deje decirle todo respeto usted e hta pero lo do buenos los pantalones bien puestos el q nesecitamos para el colombia usted representa',
 'pizarromariajo movimientomais upcolombia polodemocratico petrogustavo levyrincon aquinoticias1 betocoralg colombiahumana somospactometa pactocol cuantos delincuentes en sola imagen',
 'mal da para el duquecito en el parlamento europeo le toc quedarse calladito crey que ir paseo por el mundo nadie le iba importar lo que pasa en colombia',
 'acaba publicar una foto en barranquilla atlantico colombia httpstcozhtqmk6msv',
 'zapatilla converse talla36 la 39 euro envos nacionales en colombia httpstcohqgfo9ksbp',
 'ivanduque europarles gracias presiden

In [20]:
my_vector = HashingVectorizer()
X_counts = my_vector.fit_transform(corpus,)
tf_transformer = TfidfTransformer(use_idf=True,).fit(X_counts)
X_tf = tf_transformer.transform(X_counts)

In [23]:
X_tf

<100604x1048576 sparse matrix of type '<class 'numpy.float64'>'
	with 1957769 stored elements in Compressed Sparse Row format>

In [25]:
# Singular Value Decomposition
U, s, Vh = SparseSVD(X_tf)

In [26]:
U,s,Vh

(array([[ 1.72952343e-03,  1.23580859e-04,  2.49531531e-04,
         -2.96564322e-04,  1.98249702e-03, -6.43690925e-04],
        [-9.41047406e-05,  4.27719911e-04,  2.42690035e-04,
          4.99155709e-04,  2.76124232e-03, -1.02183962e-03],
        [ 3.69951901e-04,  8.18267991e-05,  1.47633516e-04,
         -1.83707204e-03,  3.18167119e-03, -5.43824510e-04],
        ...,
        [ 3.72340856e-03,  4.26833169e-04,  3.64286910e-04,
         -1.93136900e-03,  5.02080074e-03, -1.01081921e-03],
        [ 1.24193651e-03,  8.52687131e-04,  3.30250635e-04,
         -2.02761548e-03,  3.65075794e-03, -5.55295377e-04],
        [ 5.26976393e-03, -2.70137344e-03, -2.15101210e-03,
          1.58941063e-02,  5.85906268e-03, -1.28592282e-03]]),
 array([16.77619825, 18.51169581, 20.19059888, 20.7956491 , 38.59847036,
        64.82878603]),
 array([[-1.33059853e-05,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 1.07411306e-06,  0.00000000

In [ ]:
Vh = np.array(Vh)

In [ ]:
print(U.shape, s.shape, Vh.shape)

In [ ]:
show_topics(Vh[:n_topics])

### Calculadora de ReTweets 

In [14]:
sentences = []
for document in corpus:
    sentences.append(document.split())

In [15]:
vectors = []
for tweet in sentences:
    vectors.append(vectorized_tweet(tweet))

df['vectors'] = vectors
df.head()

KeyboardInterrupt: 

In [3]:
def greet(name):
    return "Hello " + name + "!!"


iface = gr.Interface(fn=greet, inputs="text", outputs="text")
iface.launch()

Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.


(<fastapi.applications.FastAPI at 0x1fa4e7bea30>,
 'http://127.0.0.1:7860/',
 None)